In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from datasets import Dataset
from torch.utils.data import DataLoader
import torch
from tqdm import tqdm
import json
from collections import Counter

In [2]:
"""
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
"""
# 모델과 토크나이저 로드
model_name = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained("../../../../data/jeongseokoh/hub/tokenizer/models--meta-llama--Llama-2-7b-chat-hf/snapshots/c1b0db933684edbfe29a06fa47eb19cc48025e93/")
model = AutoModelForCausalLM.from_pretrained("../../../../data/jeongseokoh/hub/model/models--meta-llama--Llama-2-7b-chat-hf/snapshots/c1b0db933684edbfe29a06fa47eb19cc48025e93/")
# 모델과 토크나이저를 동일한 장치로 이동
tokenizer.pad_token = tokenizer.eos_token


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
gen_pipe = pipeline(
        model=model, 
        tokenizer=tokenizer,
        return_full_text=True,  # langchain expects the full text
        task='text-generation',
        device=3,
        # we pass model parameters here too
        do_sample = True,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
        max_new_tokens=512,  # mex number of tokens to generate in the output
        repetition_penalty=1.1  # without this output begins repeating
        )

# Prompting

In [4]:
intent_correct_prompt_long = """<s> [INST] <<SYS>>As a language model, your role is to determine the primary purpose or goal of the user in a given dialogue. This task is known as Dialogue State Tracking. Your specific task is to read the provided dialogue and identify the user's intent. "Intent" refers to the main purpose or goal of the user in the dialogue, such as booking a hotel or finding a restaurant. \n<</SYS>>
make a response referring to intent list: [find_hotel,find_restaurant, book_train, find_attraction, find_bus, find_hospital, find_hotel, find_police, find_restaurant, find_taxi, find_train]
I'll give you few examples below. 


#### Example1 Dialogue:
<user>We are looking for staying in a place.
<user>Yes, we will be in Cambridge.
<system>We have 33 locations to stay, do you have any other requirements?<user>We can afford only moderately priced hotels.
<user>Yes, we are in the south part of the town.
<user>We should also expect free wifi.
output: find_hotel,find_restaurant,find_hospital

#### Example2 Dialogue:
<user>Could you get us a taxi?
<user>It's for a bit later.
<user>Yes, we need to get there by the late evening.
<user>Make sure we arrive by 20:00.
output: find_taxi

#### Example3 Dialogue:
<user>Should we check out the restaurant you mentioned?
<user>Yes, let's look for information about restaurant alimentum.
output: find_restaurant,find_hospital

Task Instructions: Identify the user's intent. Your response should be concise and formatted as the intent only. There is no need for an explanatory response. Simply provide your answer in the following format:
output: Your Answer
[/INST]


"""

inform_correct_prompt_long = """<s> [INST] <<SYS>>Your task is to analyze each provided dialogue and extract key information such as location, time, price range, and other pertinent details. The critical aspect of this task is to format this information in a specific style, which is "category:detail". Review the example dialogues, identify the necessary information, and present it in the designated output format.\n<</SYS>>
category list: [attraction-area, attraction-name, attraction-type, bus-day, bus-departure, bus-destination, bus-leaveat, hospital-department, hotel-area, hotel-bookday, hotel-bookpeople, hotel-bookstay, hotel-internet, hotel-name, hotel-parking, hotel-pricerange, hotel-stars, hotel-type, restaurant-area, restaurant-bookday, restaurant-bookpeople, restaurant-booktime, restaurant-food, restaurant-name, restaurant-pricerange, taxi-arriveby, taxi-departure, taxi-destination, taxi-leaveat, train-arriveby, train-bookpeople, train-day, train-departure, train-destination, train-leaveat]
I'll give you an example of scale for each task:
<domain>-day:monday,
<domain>-departure:12:00,
<domain>-arriveby:13:00,
<domain>-bookpeople:3,
<domain>-pricerange:moderate,
<domain>-food: indian,
<domain>-leaveat: 11:00,
<domain>-name: warkworth house,
<domain>-area: west


Now I'll give you few dialogue examples below. 

#### Example1 Dialogue:
<user>Can you help me find restaurant information?
<user>A restaurant for today please
<system>There are many options in Cambridge, do you have any tastes in particular?
<user>Let's look for some British food.
<user>Yes I like british.
<user>What can we find in a British restaurant?
output: <<restaurant-food:british>>

#### Example2 Dialogue:
<user>We want to go to an Indian restaurant.
<user>Yes, we'll have so much fun.
<user>Oh but there's a specific area we want it in.
<user>Oh yes, we want to be in the north.
output: <<restaurant-area:north,restaurant-food:indian>>

#### Example3 Dialogue:
<user>Let's go somewhere to eat.
<user>Do you have any place in mind?
<user>I know a restaurant known as Sala Thong.
<system>Ah yes, I have that right here. It's an expensive Thai restaurant in the west end. How many people would you like to reserve a table for, and what day/time?
<user>This is the information we needed.
<user>Yes, I think this is enough for us.
<system>Are you sure you don't need help with anything else?
<user>Is there any swimmingpool there?<user>Where are you looking for the swimming pool?
<user>I'm in in the east.
<system>Abbey pool and astroturf pitch is in the east. Would you like the address and postal code?
<user>This is great and this is what I wanted
<user>I think we would be needing the address.
<user>Yes, there is no need for the postal code.
output: <<restaurant-name:sala thong,attraction-area:east,attraction-type:swimmingpool>>

Task Instructions: Respond in the following format like <<restaurant-pricerange:moderate, ..., restaurant-area:west>>, which includes domain and category list mentioned above. identify and format the key information from the overall dialogue. Ensure all outputs strictly adhere to the "category:detail" format as illustrated in the examples. It is imperative to maintain this format for consistency in the task. Do not explain your answer. It is short-answer question!
Your response should be concise and directly formatted as required.
output: <<answer>>
[/INST]


"""
###########리퀘스트는 아직 완성 못했습니다. 프롬프트 짜고 코드도 조금 손봐야합니다. 
request_correct_prompt_long = """<s> [INST] <<SYS>>As a language model, your role is to determine the primary purpose or goal of the user in a given dialogue. This task is known as Dialogue State Tracking. Your specific task is to read the provided dialogue and identify the user's request. "Request" refers to the additional information or clarification that the user is asking for, such as availability, additional services, or specific details. \n<</SYS>>
Make a response referring to request list: [attraction-address,attraction-area,attraction-entrance fee,attraction-name,attraction-parking,attraction-phone,attraction-postcode,attraction-type,hospital-address,hospital-name,hospital-phone,hospital-postcode,hotel-address,hotel-area,hotel-internet,hotel-name,hotel-parking,hotel-phone,hotel-postcode,hotel-pricerange,hotel-ref,hotel-stars,hotel-type,police-address,police-department,police-name,police-phone,police-postcode,restaurant-address,restaurant-area,restaurant-food,restaurant-name,restaurant-phone,restaurant-postcode,restaurant-pricerange,restaurant-ref,taxi-name,taxi-phone,taxi-type,train-arriveby,train-duration,train-leaveat,train-name,train-price,train-ref,train-trainid]
I'll give you few examples below. 

<</SYS>>
#### Example1 Dialogue:
<user>Let's go somewhere to eat. I just want to eat Chinese?
<user>I will join you. Can you suggest us some Chinese restaurants that are located in the center of the town?
<system>There are 10 Chinese restaurants in the centre of town. Would you like a moderately priced one or an expensive one? We have a few cheap.
<user>I don't want to go with a cheap one. What do you save my buddy?
<user>I also agree with you on this. I think we should avoid the cheap one.
<user>So it is final that we are going ahead with an expensive one.
<system>I have found tang Chinese which is here in the centre. Would you like to start a reservation?
<user>If it fulfills our criteria, I think we should go ahead with the booking. What do you say on this?
<user>I think we should go ahead with the booking. I forgot to tell you that two of my friends are also joining us.
<user>So that would make us a total of 4 people. Is Friday fine for you?
<user>Yes, Friday is good for us. So, please make a reservation for four people at 16:15 on Friday. Please save the reference number at the earliest.
Request: restaurant-ref

#### Example2 Dialogue:
<user>we are looking for a place to go
<user>yes, an attraction in cambridge
<user>perhaps an architecture attraction
<system>There are 5 architecture attractions in the centre. WIll one of those work?
<user>i'm sure one will work
<user>what would you suggest?
<user>yes, could you please recommend one for us
Request: attraction-name

#### Example3 Dialogue:
<user>We are going to Cambridge.
<user>Yes, we want to try the local restaurants there.
<user>But we will also need a place to stay.
<user>Yes we would need a hotel.
<system>There are many restaurants. Can you please elaborate on what you would like?
<user>First, we will need a hotel with a particular rating.
<user>Yes, we are looking for something 3 stars.
<user>We would need internet as well.
<user>Yes, so we need to make sure that it has free wifi.
<system>I have 5 options for you, located all over town. Do you have a certain area or price range in mind?
<user>We would like some cheap price range place with chinese food.
<user>Yes, we will be in the west part of the town.
<user>We would need an expensive hotel with 3 stars and wifi.
<user>Yes, we would need the hotel address, area and postcode as well.
Request: hotel-area,hotel-address

Task Instructions: Identify the user's request. Your response should be concise and formatted as the request only. Do not explain your answer. It is a short-answer question. There is no need for an explanatory response. Simply provide your answer in the following format:
Request: Your Answer
"""
len(inform_correct_prompt_long)

3311

# Intent Model Evaluation

In [13]:
with open("intent_test_data.json", "r") as file:
    test_dataset = json.load(file)

In [17]:
# 모델을 평가하기 위한 함수
def evaluate(model, tokenizer, dataset, device='cuda:3'):
    y_pred = []
    y_real = []
    model.to(device)
    input_data = []
    output_data = []
    for text in dataset["input"]:
        input_data.append(text)
    for text in dataset["output"]:
        output_data.append(text)
    for text in output_data:
        out = text.split(",")
        y_real.append(out)
    #print(y_real)
    tp, fn, fp = 0, 0, 0
    for i in tqdm(range(len(dataset['input']))):
        # 입력 프롬프트 생성
        input_text = f"{intent_correct_prompt_long}\n{input_data[i]}\n"              #intent_prompt 앞부분에 concat
        generated_answer = gen_pipe(input_text)[0]['generated_text'][1600:]
        #print(f'OUTPUT_TEXT: {generated_answer}\n')


        #print(f'output_size: {output_size}')
        gen_start_id = generated_answer.find("put:") #### 1570번째 붙이는 이유: llama2는 input을 그대로 뱉고 output을 주더라고요. 앞에 input이 대략 1570자가 있다고 보시면 됩니다.
        #print(generated_answer[1377+gen_start_id:])
        real_gen = generated_answer[4+gen_start_id:].strip().split(",") #strip 사용해야함! ## put: 이 4자라서 4를 더해서 1574입니다. 
        #print(f'real_gen: {real_gen}')
        y_pred.append(real_gen)

    #true_positive, false_positive, false_negative 계산    
    for i, real in enumerate(y_real):
        #print(f'real_y: {real}')
        #print(f'pred_y: {y_pred[i]}')
        real_cnt = Counter(real) 
        pred_cnt = Counter(y_pred[i])
        common = real_cnt & pred_cnt
        tp += len(common.values())
        if len(real) > len(y_pred[i]):
            gap = len(real) - len(common.values())
            fn += gap
        elif len(real) < len(y_pred[i]):
            gap = len(y_pred[i]) - len(common.values())
            fp += gap
    print(f'tp: {tp}, fp: {fp}, fn: {fn}')
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * (precision*recall/(precision+recall))

    metric = [precision, recall, f1]
    return metric

# 테스트 데이터셋에 대한 모델 평가
accuracy = evaluate(model, tokenizer, test_dataset)
print(f'F1: {accuracy[2]}\nPrecision: {accuracy[0]}\nRecall: {accuracy[1]}')

100%|██████████| 2869/2869 [35:43<00:00,  1.34it/s] 

tp: 1956, fp: 797, fn: 356
F1: 0.7723593287265548
Precision: 0.7104976389393389
Recall: 0.8460207612456747


# Inform Model Evaluation

In [6]:
with open("inform_test_data.json", "r") as file:
    test_dataset = json.load(file)

In [7]:
# 모델을 평가하기 위한 함수

def evaluate(dataset):
    y_pred = []
    y_real = []
    input_data = []
    output_data = []
    for text in dataset["input"][:10]:
        input_data.append(text)
    for text in dataset["output"][:10]:
        output_data.append(text)
    for text in output_data:
        out = text.split(",")
        y_real.append(out)
    #print(y_real)
    tp, fn, fp = 0, 0, 0
    for i in tqdm(range(len(dataset['input'][:10]))):
        # 입력 프롬프트 생성
        input_text = f"{inform_correct_prompt_long}\n{input_data[i]}\n"              #intent_prompt 앞부분에 concat
        generated_answer = gen_pipe(input_text)[0]['generated_text'][3311:]
        #print(f'OUTPUT_TEXT: {generated_answer}\n')


        #print(f'output_size: {output_size}')
        if "<<" in generated_answer:
            gen_start_id = generated_answer.find("<<")
            gen_end_id = generated_answer.find(">>") 
            #print(generated_answer[gen_start_id:])
            real_gen = generated_answer[2+gen_start_id:gen_end_id].strip().split(",") 
            #print(f'real_gen: {real_gen}')
        else:
            real_gen = ""
        y_pred.append(real_gen)             
        

    #true_positive, false_positive, false_negative 계산    
    for i, real in enumerate(y_real):
        print(f'real_y: {real}')
        print(f'pred_y: {y_pred[i]}')
        real_cnt = Counter(real) 
        pred_cnt = Counter(y_pred[i])
        common = real_cnt & pred_cnt
        tp += len(common.values())
        if len(real) > len(y_pred[i]):
            gap = len(real) - len(common.values())
            fn += gap
        elif len(real) < len(y_pred[i]):
            gap = len(y_pred[i]) - len(common.values())
            fp += gap
    print(f'tp: {tp}, fp: {fp}, fn: {fn}')
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * (precision*recall/(precision+recall))

    metric = [precision, recall, f1]
    return metric

# 테스트 데이터셋에 대한 모델 평가
accuracy = evaluate(test_dataset)
print(f'F1: {accuracy[2]}\nPrecision: {accuracy[0]}\nRecall: {accuracy[1]}')

100%|██████████| 10/10 [00:40<00:00,  4.09s/it]

real_y: ['']
pred_y: ['hotel-name:warkworth house', 'hotel-price-range:moderate', 'hotel-location:cambridge', 'hotel-distance-from-station:5-minutes']
real_y: ['hotel-area:south', 'hotel-internet:yes', 'hotel-pricerange:moderate']
pred_y: ['hotel-pricerange:moderate', 'hotel-area:south', 'hotel-wifi:yes']
real_y: ['hotel-area:south', 'hotel-internet:yes', 'hotel-pricerange:moderate']
pred_y: ['hotel-name:bridge guest house', 'hotel-pricerange:moderate', 'hotel-area:south', 'hotel-wifi:yes']
real_y: ['restaurant-area:south', 'restaurant-food:chinese', 'hotel-area:south', 'hotel-internet:yes', 'hotel-pricerange:moderate']
pred_y: ['hotel-name:bridge guest house', 'hotel-pricerange:moderate', 'hotel-area:south', 'hotel-parking:yes', 'attraction-name:chinese restaurant', 'attraction-area:near hotel']
real_y: ['restaurant-name:restaurant alimentum']
pred_y: ['restaurant-name:alimentum', 'attraction-distance:2mi', 'attraction-area:citycenter', 'attraction-type:italianrestaurant', 'diningtime

# Request Model Evaluation

In [5]:
with open("request_test_data.json", "r") as file:
    test_dataset = json.load(file)

In [27]:
# 모델을 평가하기 위한 함수

def evaluate(dataset):
    y_pred = []
    y_real = []
    input_data = []
    output_data = []
    for text in dataset["input"]:
        input_data.append(text)
    for text in dataset["output"]:
        output_data.append(text)
    for text in output_data:
        out = text.split(",")
        y_real.append(out)
    #print(y_real)
    tp, fn, fp = 0, 0, 0
    for i in tqdm(range(len(dataset['input']))):
        # 입력 프롬프트 생성
        input_text = f"{request_correct_prompt_long}\n{input_data[i]}\n"              #intent_prompt 앞부분에 concat
        generated_answer = gen_pipe(input_text)[0]['generated_text'][3860:]
        #print(f'OUTPUT_TEXT: {generated_answer}\n')

        
        #print(f'output_size: {output_size}')
        if "put:" in generated_answer:
            gen_start_id = generated_answer.find("put:")
            print(generated_answer[gen_start_id:])
            real_gen = generated_answer[4+gen_start_id:].replace("*","").replace("\n","").strip().split(",") 
            #print(f'real_gen: {real_gen}')
        elif "est:" in generated_answer:
            gen_start_id = generated_answer.find("est:")
            #print(generated_answer[gen_start_id:])
            real_gen = generated_answer[4+gen_start_id:].replace("*","").replace("\n","").strip().split(",") 
        else:
            real_gen = [""]
        y_pred.append(real_gen)             
        

    #true_positive, false_positive, false_negative 계산    
    for i, real in enumerate(y_real):
        #print(f'real_y: {real}')
        #print(f'pred_y: {y_pred[i]}')
        real_cnt = Counter(real) 
        pred_cnt = Counter(y_pred[i])
        common = real_cnt & pred_cnt
        tp += len(common.values())
        if len(real) > len(y_pred[i]):
            gap = len(real) - len(common.values())
            fn += gap
        elif len(real) < len(y_pred[i]):
            gap = len(y_pred[i]) - len(common.values())
            fp += gap
    print(f'tp: {tp}, fp: {fp}, fn: {fn}')
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * (precision*recall/(precision+recall))

    metric = [precision, recall, f1]
    return metric

# 테스트 데이터셋에 대한 모델 평가
accuracy = evaluate(test_dataset)
print(f'F1: {accuracy[2]}\nPrecision: {accuracy[0]}\nRecall: {accuracy[1]}')

  0%|          | 0/20 [00:00<?, ?it/s]/home/jeongseokoh/miniconda3/lib/python3.11/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
 10%|█         | 2/20 [00:02<00:22,  1.27s/it]

est: hotel-pricerange


 15%|█▌        | 3/20 [00:03<00:20,  1.24s/it]

est: hotel-postcode


 20%|██        | 4/20 [00:05<00:21,  1.32s/it]

...

est: restaurant-ref


 95%|█████████▌| 19/20 [00:24<00:01,  1.30s/it]

est: restaurant-name


100%|██████████| 20/20 [00:25<00:00,  1.29s/it]

est: restaurant-food
real_y: ['']
pred_y: ['']
real_y: ['']
pred_y: ['hotel-pricerange']
real_y: ['hotel-address', 'hotel-parking']
pred_y: ['hotel-postcode']
real_y: ['']
pred_y: ['hotel-name', 'restaurant-name']
real_y: ['']
pred_y: ['restaurant-name']
real_y: ['']
pred_y: ['restaurant-name']
real_y: ['']
pred_y: ['']
real_y: ['']
pred_y: ['taxi-name']
real_y: ['']
pred_y: ['taxi-name']
real_y: ['']
pred_y: ['taxi-name', 'taxi-phone']
real_y: ['']
pred_y: ['restaurant-ref']
real_y: ['']
pred_y: ['restaurant-booking']
real_y: ['']
pred_y: ['attraction-address']
real_y: ['']
pred_y: ['attraction-name']
real_y: ['']
pred_y: ['restaurant-ref']
real_y: ['']
pred_y: ['restaurant-ref']
real_y: ['restaurant-ref']
pred_y: ['restaurant-ref']
real_y: ['']
pred_y: ['']
real_y: ['']
pred_y: ['restaurant-name']
real_y: ['']
pred_y: ['restaurant-food']
tp: 4, fp: 4, fn: 2
F1: 0.5714285714285715
Precision: 0.5
Recall: 0.6666666666666666
